# Calculating Historical Performance for Unique Flights (An Optimized Approach)

Each row in the flights dataset represents a unique flight, containing details such as flight date, carrier, origin, destination, scheduled departure time, arrival time, and performance metrics like delays and cancellations. To extract meaningful historical insights for each flight, I performed rolling window calculations to gather statistics based on recent flight history over a certain period of time. The idea for this aggregation approach to create new features was adapted from [Ashish Jain](https://github.com/aajains/springboard-datascience-intensive/blob/master/capstone_project/DataAcquisitionMerging_v1/data_acquisition_merging.ipynb).

#### Objective:
I wanted to compute historical performance statistics—such as delays, cancellations, and diversions—for each flight, using its recent history. For example, for a flight from Las Vegas (LAS) to Charlotte (CLT) on August 6th, 2023, at 11:59 PM, I wanted to know how many times a flight on the same route, operated by the same airline, and within the same time window, was delayed or canceled over the past 10 days, 20 days, 30 days and 90 days.

#### Key Variables Defining a Unique Flight:
1. **Carrier**: The airline operating the flight.
2. **Origin**: The departure airport.
3. **Destination**: The arrival airport.
4. **Departure Window**: A specific time range during which the flight departs (e.g., morning, afternoon).

For each flight, I gathered historical data based on these four variables.

#### Process Overview:
1. **Grouping by Flight Attributes**: First step is to group flights by their carrier, route (origin-destination pair), and departure window. 
    * Use the function `create_route_ids()` to create a new dataframe containing unique routes made up of origin destination pairs and assign each route a `route_id`. This id was then added to each flight in the flights dataframe. 
    * Use the function `create_time_windows()` to create a new categorical column called `departure_window` with the categories and times below. These windows are based on the scheduled departure time in the local timezone. 

        | Category         | Time Range    |
        |------------------|---------------|
        | Overnight        | 12 AM - 4 AM  |
        | Early morning    | 4 AM - 6 AM   |
        | Morning          | 6 AM - 11 AM  |
        | Midday           | 11 AM - 1 PM  |
        | Early afternoon  | 1 PM - 3 PM   |
        | Afternoon        | 3 PM - 5 PM   |
        | Evening          | 5 PM - 7 PM   |
        | Night            | 7 PM - 10 PM  |
        | Late night       | 10 PM - 12 AM |


   
2. **Rolling Window Aggregations**: Next, for each flight, use the `calculate_flight_performance_aggregations(df, windows)` function calculate performance statistics (e.g., delay metrics, cancellations, and diversions) over rolling time windows (e.g., 10, 20, 30, and 90 days) that look back from the flight's scheduled departure date. The statistics include:
   - Median and maximum departure/arrival delays
   - Count of canceled flights
   - Count of diversions
   - Count of flights

3. **Handling Duplicate Data**: The aggregations are done for all specified time windows for each flight in the flights dataset. Some duplicates occurred when an airline had multiple flights on the same route and in the same time window. This caused the merge with the original flights dataset to result in more rows than expected, as one flight could be matched with multiple rows of aggregated data. After computing the rolling window statistics, I optimized the deduplication process by sorting the aggregated dataframe by key columns (route_id, airline_mkt, departure_window, scheduled_departure_datetime) and keeping the row with the highest number of n_flights. This ensured that for each combination of key columns, the row with the most complete flight data (i.e., the highest number of flights) is retained.

This deduplication was applied with the function, `drop_agg_duplicates()` before merging the rolling statistics back into the original flights dataframe.

4. **Time Windows**: Flights departing at similar times are grouped into “time windows” (e.g., early morning, afternoon), allowing for tracking consistent patterns in flight operations over time. This reduces the granularity of individual departure times and simplifies grouping.

#### Optimized Approach:
Rather than looping through each flight row by row, I leveraged vectorized operations with `pandas` to group flights by carrier, route, and departure window and applied rolling windows efficiently. This dramatically reduces computation time and allows the process to effectively scale large datasets with millions of rows. After calculating the aggregations, I merged the results back into the original flight dataset.

By combining these rolling window statistics with the original flight data, I enhanced the dataset with valuable historical features that can be used for further analysis and modeling.

#### Future Steps for EDA
1. **Handling Rows with `n_flights == 0` or `n_flights == NaN`**

    During the rolling window aggregation process, some rows resulted in `n_flights == 0`, indicating that there were no relevant flights within the specified window for the given route, carrier, and departure window. This can happen for several reasons:
    * Sparse Data: Some routes or carriers may not have had enough flights during the rolling window, especially for less common routes or carriers.
    * Edge Cases: Flights near the start or end of the dataset may not have enough prior or subsequent data to fill the rolling window (example flights in the first 10 days of the year)

    Rows with `n_flights == 0` often have NaN values for other aggregated columns (e.g., delays or cancellations), as no data was available to compute these statistics.

    *Handling Strategy:*
    * Preserving Data: Rather than dropping rows with `n_flights == 0`, I preserved them in the dataset. These rows will be handled during Exploratory Data Analysis (EDA) or preprocessing steps, where I can choose to filter them out, impute missing values, or flag them for special handling, depending on the analysis needs.
    * Analysis Consideration: These rows are important to consider because they reflect scenarios where there is limited historical data for specific flights, which can impact the accuracy of predictions or analysis. By keeping them, I maintain transparency in the data and can address any gaps during the modeling process.

In [1]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)

In [2]:
import pandas as pd
import numpy as np

from config.config import DATA_PATH


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
flights = pd.read_csv(DATA_PATH + '/interim/2023-performance-data-clean.csv',  parse_dates=['scheduled_departure_datetime'])

In [5]:
flights.head()

,year,quarter,month,day_of_month,day_of_week,marketing_airline_id,flight_number_marketing_airline,operating_airline_id,tail_number,origin_airport_id,origin_city_market_id,origin,origin_state,dest_airport_id,dest_city_market_id,dest,dest_state,dep_delay,dep_del15,departure_delay_groups,taxi_out,taxi_in,arr_delay,arr_del15,arrival_delay_groups,cancelled,diverted,scheduled_elapsed_time,actual_elapsed_time,air_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,div_airport_landings,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,actual_departure_datetime,actual_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,scheduled_departure_datetime_utc,actual_departure_datetime_utc,scheduled_arrival_datetime_utc,actual_arrival_datetime_utc,is_holiday
0,2023,3,8,6,7,20416,2252,20416,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,263.0,NaN,NaN,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-06 23:59:00,2023-08-07 07:22:00,NaN,NaN,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-07 07:00:00+00:00,NaN,2023-08-07 11:00:00+00:00,NaN,0
1,2023,3,8,7,1,20416,2252,20416,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,76.0,1.0,5.0,19.0,9.0,69.0,1.0,4.0,0,0,263.0,256.0,228.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,0,Las Vegas,Charlotte,2023-08-07 23:59:00,2023-08-08 07:22:00,2023-08-08 01:15:00,2023-08-08 08:31:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-08 07:00:00+00:00,2023-08-08 08:15:00+00:00,2023-08-08 11:00:00+00:00,2023-08-08 12:31:00+00:00,0
2,2023,3,8,9,3,20416,2252,20416,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-11.0,0.0,-1.0,14.0,10.0,-13.0,0.0,-1.0,0,0,258.0,256.0,232.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-09 23:10:00,2023-08-10 06:28:00,2023-08-09 22:59:00,2023-08-10 06:15:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-10 06:00:00+00:00,2023-08-10 05:59:00+00:00,2023-08-10 10:00:00+00:00,2023-08-10 10:15:00+00:00,0
3,2023,3,8,10,4,20416,2252,20416,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-8.0,0.0,-1.0,12.0,7.0,-30.0,0.0,-2.0,0,0,258.0,236.0,217.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-10 23:10:00,2023-08-11 06:28:00,2023-08-10 23:02:00,2023-08-11 05:58:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-11 06:00:00+00:00,2023-08-11 06:02:00+00:00,2023-08-11 10:00:00+00:00,2023-08-11 09:58:00+00:00,0
4,2023,3,8,12,6,20416,2252,20416,N529NK,12889,32211,LAS,NV,11057,31057,CLT,NC,4.0,0.0,0.0,18.0,6.0,-4.0,0.0,-1.0,0,0,258.0,250.0,226.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-12 23:10:00,2023-08-13 06:28:00,2023-08-12 23:14:00,2023-08-13 06:24:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-13 06:00:00+00:00,2023-08-13 06:14:00+00:00,2023-08-13 10:00:00+00:00,2023-08-13 10:24:00+00:00,0


In [6]:
flights.shape

(7260941, 56)

In [7]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7260941 entries, 0 to 7260940
Data columns (total 56 columns):
 #   Column                            Dtype         
---  ------                            -----         
 0   year                              int64         
 1   quarter                           int64         
 2   month                             int64         
 3   day_of_month                      int64         
 4   day_of_week                       int64         
 5   marketing_airline_id              int64         
 6   flight_number_marketing_airline   int64         
 7   operating_airline_id              int64         
 8   tail_number                       object        
 9   origin_airport_id                 int64         
 10  origin_city_market_id             int64         
 11  origin                            object        
 12  origin_state                      object        
 13  dest_airport_id                   int64         
 14  dest_city_market_i

In [8]:
dep_delay_99th = flights['dep_delay'].quantile(0.99)
arr_delay_99th = flights['arr_delay'].quantile(0.99)

# Clip values at the 99th percentile
flights['dep_delay_clipped'] = flights['dep_delay'].clip(upper=dep_delay_99th)
flights['arr_delay_clipped'] = flights['arr_delay'].clip(upper=arr_delay_99th)

In [9]:
def create_route_ids(df):
    """
    Assign unique route IDs to each origin and destination airport pair.

    Parameters:
    ----------
    flights : pandas.DataFrame
        The original DataFrame of flight data, which must contain 'origin' and 'dest' columns.

    Returns:
    -------
    flights : pandas.DataFrame
        The DataFrame with two new columns:
        - 'route_id': A unique identifier for each route (origin-destination pair).
    """

    routes = df.groupby(['origin', 'dest'])[['origin', 'dest']].size().reset_index(name='flight_count')
    routes.index += 1
    routes = routes.reset_index().rename(columns={'index':'route_id'})

    # create a dictionary with route tuples and route id
    route_mapping = routes.set_index(['origin', 'dest'])['route_id'].to_dict()

    # add the route id for each flight
    df['route_id'] = df[['origin', 'dest']].apply(tuple, axis=1).map(route_mapping)

    return df, route_mapping

In [10]:
flights, routes = create_route_ids(flights)

In [11]:
flights['route_id'].nunique()

6714

In [12]:
def create_time_windows(df, datetime_col):
    """
    Categorize flights into departure time windows.

    Time windows (e.g., morning, afternoon) are assigned based on the hour of day using 
    predefined bins.

    Parameters:
    ----------
    flights : pandas.DataFrame
        The original DataFrame of flight data, which must contain a datetime column to 
        convert such as the 'scheduled_departure_datetime' column.

    Returns:
    -------
    flights : pandas.DataFrame
        The DataFrame with two new columns:
        - 'departure_window': A categorical time window label (e.g., morning, afternoon).
    """

    bins = [0, 4, 6, 11, 13, 15, 17, 19, 22, 24]
    labels = ['overnight', 'early morning', 'morning', 'midday', 'early afternoon', 'afternoon', 'evening', 'night', 'late night']

    df['hour_of_day'] = df[datetime_col].dt.hour

    df['departure_window'] = pd.cut(df['hour_of_day'], bins=bins, labels=labels, right=False)

    return df

In [13]:
flights = create_time_windows(flights, 'scheduled_departure_datetime')

In [14]:
flights.head()

,year,quarter,month,day_of_month,day_of_week,marketing_airline_id,flight_number_marketing_airline,operating_airline_id,tail_number,origin_airport_id,origin_city_market_id,origin,origin_state,dest_airport_id,dest_city_market_id,dest,dest_state,dep_delay,dep_del15,departure_delay_groups,taxi_out,taxi_in,arr_delay,arr_del15,arrival_delay_groups,cancelled,diverted,scheduled_elapsed_time,actual_elapsed_time,air_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,div_airport_landings,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,actual_departure_datetime,actual_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,scheduled_departure_datetime_utc,actual_departure_datetime_utc,scheduled_arrival_datetime_utc,actual_arrival_datetime_utc,is_holiday,dep_delay_clipped,arr_delay_clipped,route_id,hour_of_day,departure_window
0,2023,3,8,6,7,20416,2252,20416,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,263.0,NaN,NaN,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-06 23:59:00,2023-08-07 07:22:00,NaN,NaN,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-07 07:00:00+00:00,NaN,2023-08-07 11:00:00+00:00,NaN,0,NaN,NaN,3228,23,late night
1,2023,3,8,7,1,20416,2252,20416,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,76.0,1.0,5.0,19.0,9.0,69.0,1.0,4.0,0,0,263.0,256.0,228.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,0,Las Vegas,Charlotte,2023-08-07 23:59:00,2023-08-08 07:22:00,2023-08-08 01:15:00,2023-08-08 08:31:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-08 07:00:00+00:00,2023-08-08 08:15:00+00:00,2023-08-08 11:00:00+00:00,2023-08-08 12:31:00+00:00,0,76.0,69.0,3228,23,late night
2,2023,3,8,9,3,20416,2252,20416,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-11.0,0.0,-1.0,14.0,10.0,-13.0,0.0,-1.0,0,0,258.0,256.0,232.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-09 23:10:00,2023-08-10 06:28:00,2023-08-09 22:59:00,2023-08-10 06:15:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-10 06:00:00+00:00,2023-08-10 05:59:00+00:00,2023-08-10 10:00:00+00:00,2023-08-10 10:15:00+00:00,0,-11.0,-13.0,3228,23,late night
3,2023,3,8,10,4,20416,2252,20416,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-8.0,0.0,-1.0,12.0,7.0,-30.0,0.0,-2.0,0,0,258.0,236.0,217.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-10 23:10:00,2023-08-11 06:28:00,2023-08-10 23:02:00,2023-08-11 05:58:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-11 06:00:00+00:00,2023-08-11 06:02:00+00:00,2023-08-11 10:00:00+00:00,2023-08-11 09:58:00+00:00,0,-8.0,-30.0,3228,23,late night
4,2023,3,8,12,6,20416,2252,20416,N529NK,12889,32211,LAS,NV,11057,31057,CLT,NC,4.0,0.0,0.0,18.0,6.0,-4.0,0.0,-1.0,0,0,258.0,250.0,226.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-12 23:10:00,2023-08-13 06:28:00,2023-08-12 23:14:00,2023-08-13 06:24:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-13 06:00:00+00:00,2023-08-13 06:14:00+00:00,2023-08-13 10:00:00+00:00,2023-08-13 10:24:00+00:00,0,4.0,-4.0,3228,23,late night


In [65]:
def calculate_flight_performance_aggregations(df, windows):
    """
    Calculate rolling window flight performance aggregations over specified time windows.

    This function calculates statistics like mean, median, max of departure and arrival delays,
    as well as the number of cancellations, diversions, and the number of flights, for each 
    flight based on its carrier, route, and departure window.

    Parameters:
    ----------
    df : pandas.DataFrame
        The input DataFrame containing flight data, including columns such as 'airline_mkt', 
        'route_id', 'departure_window', and 'scheduled_departure_datetime'.
        
    windows : list of str
        A list of rolling window durations (e.g., ['10D', '20D', '30D']) to calculate the
        rolling aggregations over. These should be formatted as Pandas-compatible time strings.

    Returns:
    -------
    pandas.DataFrame
        A DataFrame containing the rolling statistics for each window. Each statistic will be 
        suffixed with the corresponding window (e.g., 'dep_delay_mean_10D', 'n_flights_20D').
        The resulting DataFrame includes statistics like:
        - dep_delay_mean, dep_delay_median, dep_delay_max
        - arr_delay_mean, arr_delay_median, arr_delay_max
        - cancelled_sum, div_airport_landings_sum
        - n_flights (number of flights in the window)
    """
    # Sort dataframe by scheduled departure datetime column
    df = df.sort_values('scheduled_departure_datetime')

    df['dep_delay'] = df['dep_delay_clipped']
    df['arr_delay'] = df['arr_delay_clipped']

    # Group dataframe to create unique flights based on airline, route, and departure window
    grouped_df = df.groupby(['airline_mkt', 'route_id', 'departure_window'], observed=False)

    # Initiate empty list to capture aggregate values
    rolling_stats_list = []

    # Iterate through unique flight groupings to calculate performance metrics over rolling window
    for window in windows:
        rolling_stats = grouped_df.rolling(window=window, on='scheduled_departure_datetime').agg({
        'dep_delay': ['mean', 'median', 'max'],
        'arr_delay': ['mean', 'median', 'max'],
        'cancelled': 'sum',
        'div_airport_landings': 'sum'
        }).reset_index()

        # Add the number of flights by counting rows within each rolling window
        rolling_stats[f'n_flights_{window}'] = grouped_df.rolling(window=window, on='scheduled_departure_datetime')['dep_delay'].count().values

        # Flatten the multi-level column index for easier access
        rolling_stats.columns = [
            f'{col[0]}_{col[1]}_{window}' if isinstance(col, tuple) and col[1] != '' else col[0] for col in rolling_stats.columns
        ]

        rolling_stats_list.append(rolling_stats)

    # Concatenate all rolling stats into a single DataFrame
    all_rolling_stats = pd.concat(rolling_stats_list, axis=1)

    # Remove duplicate columns from the concatenation (from the index or key columns)
    all_rolling_stats = all_rolling_stats.loc[:, ~all_rolling_stats.columns.duplicated()]

    return all_rolling_stats    


In [66]:
# calculate rolling statistics for 2023 unique flights
rolling_stats = calculate_flight_performance_aggregations(flights, ['10D', '30D', '90D'])

In [67]:
rolling_stats.shape

(7260941, 31)

Initially I merged `rolling_stats` with the flights dataframe but due to duplicates, the merged dataframe was several thousand rows longer. The code below is how I investigated the duplicates. 

In [68]:
rolling_stats.duplicated(subset=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime']).sum()

152

In [69]:
rolling_stats[rolling_stats.duplicated(subset=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime'], keep=False)].sort_values('scheduled_departure_datetime').head()


,airline_mkt,route_id,departure_window,scheduled_departure_datetime,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_mean_30D,arr_delay_median_30D,arr_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,n_flights_30D,dep_delay_mean_90D,dep_delay_median_90D,dep_delay_max_90D,arr_delay_mean_90D,arr_delay_median_90D,arr_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,n_flights_90D
3840642,Frontier Airlines Inc.,3781,early morning,2023-03-05 05:45:00,-0.300000,-2.0,20.0,-6.500000,-8.0,18.0,0.0,0.0,10.0,0.722222,-2.0,39.0,-3.277778,-7.0,40.0,0.0,0.0,18.0,0.722222,-2.0,39.0,-3.277778,-7.0,40.0,0.0,0.0,18.0
3840643,Frontier Airlines Inc.,3781,early morning,2023-03-05 05:45:00,-0.181818,-2.0,20.0,-5.363636,-8.0,18.0,0.0,0.0,11.0,0.736842,-2.0,39.0,-2.789474,-7.0,40.0,0.0,0.0,19.0,0.736842,-2.0,39.0,-2.789474,-7.0,40.0,0.0,0.0,19.0
2495370,Delta Air Lines Inc.,288,morning,2023-03-14 09:10:00,-0.100000,-1.0,11.0,-0.200000,-3.0,20.0,1.0,0.0,10.0,3.310345,-1.0,99.0,3.344828,-2.0,101.0,2.0,0.0,29.0,4.805556,-1.0,113.0,-2.458333,-4.0,104.0,2.0,0.0,72.0
2495369,Delta Air Lines Inc.,288,morning,2023-03-14 09:10:00,0.111111,0.0,11.0,0.888889,-3.0,20.0,1.0,0.0,9.0,3.500000,-1.0,99.0,3.821429,-1.0,101.0,2.0,0.0,28.0,4.901408,-1.0,113.0,-2.352113,-4.0,104.0,2.0,0.0,71.0
3676425,Delta Air Lines Inc.,6191,midday,2023-03-14 12:40:00,13.400000,2.0,104.0,0.200000,-10.5,114.0,1.0,0.0,10.0,10.275862,0.0,106.0,1.655172,-5.0,114.0,2.0,0.0,29.0,6.464789,-2.0,108.0,2.380282,-1.0,114.0,2.0,0.0,71.0


In order to address the issue of one flight getting paired with two unique flight values, I decided to drop duplicates and keep the last value. 

In [70]:
def drop_agg_duplicates(df):
    """
    Remove duplicate rows from the aggregated flight performance data while keeping the row 
    with the highest number of flights (n_flights) for each group.

    The DataFrame is sorted by 'route_id', 'airline_mkt', 'departure_window', and 
    'scheduled_departure_datetime', and duplicates are dropped based on these columns. 
    The row with the highest 'n_flights' for each group is retained.

    Parameters:
    ----------
    df : pandas.DataFrame
        The input DataFrame containing the rolling flight performance statistics, 
        including the columns 'route_id', 'airline_mkt', 'departure_window', 
        'scheduled_departure_datetime', and 'n_flights_<window>'.

    Returns:
    -------
    pandas.DataFrame
        A deduplicated DataFrame where only one row per unique combination of 'route_id', 
        'airline_mkt', 'departure_window', and 'scheduled_departure_datetime' is retained,
        specifically the row with the highest 'n_flights'.
    """
    df_sorted = df.sort_values( ['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime', 'n_flights_90D'], ascending=True)

    df_deduped = df_sorted.drop_duplicates(subset=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime'], keep='last')

    return df_deduped

In [71]:
rolling_stats_deduped = drop_agg_duplicates(rolling_stats)

In [72]:
rolling_stats_deduped.shape

(7260789, 31)

In [73]:
flights_historical_performance = pd.merge(flights, rolling_stats_deduped, 
    on=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime'], 
    how='left')

In [74]:
flights_historical_performance.isna().sum()

year                               0
quarter                            0
month                              0
day_of_month                       0
day_of_week                        0
                                ... 
arr_delay_median_90D            1076
arr_delay_max_90D               1076
cancelled_sum_90D                  0
div_airport_landings_sum_90D       0
n_flights_90D                      0
Length: 88, dtype: int64

In [75]:
flights_historical_performance.head()

,year,quarter,month,day_of_month,day_of_week,marketing_airline_id,flight_number_marketing_airline,operating_airline_id,tail_number,origin_airport_id,origin_city_market_id,origin,origin_state,dest_airport_id,dest_city_market_id,dest,dest_state,dep_delay,dep_del15,departure_delay_groups,taxi_out,taxi_in,arr_delay,arr_del15,arrival_delay_groups,cancelled,diverted,scheduled_elapsed_time,actual_elapsed_time,air_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,div_airport_landings,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,actual_departure_datetime,actual_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,scheduled_departure_datetime_utc,actual_departure_datetime_utc,scheduled_arrival_datetime_utc,actual_arrival_datetime_utc,is_holiday,dep_delay_clipped,arr_delay_clipped,route_id,hour_of_day,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_mean_30D,arr_delay_median_30D,arr_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,n_flights_30D,dep_delay_mean_90D,dep_delay_median_90D,dep_delay_max_90D,arr_delay_mean_90D,arr_delay_median_90D,arr_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,n_flights_90D
0,2023,3,8,6,7,20416,2252,20416,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,263.0,NaN,NaN,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-06 23:59:00,2023-08-07 07:22:00,NaN,NaN,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-07 07:00:00+00:00,NaN,2023-08-07 11:00:00+00:00,NaN,0,NaN,NaN,3228,23,late night,5.125000,2.0,31.0,-5.000000,-6.0,16.0,3.0,0.0,8.0,15.518519,5.0,82.0,4.923077,0.5,62.0,4.0,0.0,27.0,15.714286,6.0,163.0,4.903614,-2.0,166.0,5.0,0.0,84.0
1,2023,3,8,7,1,20416,2252,20416,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,76.0,1.0,5.0,19.0,9.0,69.0,1.0,4.0,0,0,263.0,256.0,228.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,0,Las Vegas,Charlotte,2023-08-07 23:59:00,2023-08-08 07:22:00,2023-08-08 01:15:00,2023-08-08 08:31:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-08 07:00:00+00:00,2023-08-08 08:15:00+00:00,2023-08-08 11:00:00+00:00,2023-08-08 12:31:00+00:00,0,76.0,69.0,3228,23,late night,14.000000,2.0,76.0,4.714286,-6.0,69.0,3.0,0.0,8.0,17.740741,5.0,82.0,7.961538,5.0,69.0,4.0,0.0,27.0,16.690476,7.0,163.0,5.819277,-1.0,166.0,5.0,0.0,84.0
2,2023,3,8,9,3,20416,2252,20416,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-11.0,0.0,-1.0,14.0,10.0,-13.0,0.0,-1.0,0,0,258.0,256.0,232.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-09 23:10:00,2023-08-10 06:28:00,2023-08-09 22:59:00,2023-08-10 06:15:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-10 06:00:00+00:00,2023-08-10 05:59:00+00:00,2023-08-10 10:00:00+00:00,2023-08-10 10:15:00+00:00,0,-11.0,-13.0,3228,23,late night,12.375000,1.5,76.0,5.000000,-6.0,69.0,3.0,0.0,8.0,15.185185,5.0,82.0,6.192308,0.5,69.0,4.0,0.0,27.0,16.710843,7.0,163.0,5.573171,-2.0,166.0,5.0,0.0,83.0
3,2023,3,8,10,4,20416,2252,20416,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-8.0,0.0,-1.0,12.0,7.0,-30.0,0.0,-2.0,0,0,258.0,236.0,217.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-10 23:10:00,2023-08-11 06:28:00,2023-08-10 23:02:00,2023-08-11 05:58:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-11 06:00:00+00:00,2023-08-11 06:02:00+00:00,2023-08-11 10:00:00+00:00,2023-08-11 09:58:00+00:00,0,-8.0,-30.0,3228,23,late night,10.111111,1.0,76.0,0.625000,-8.5,69.0,2.0,0.0,9.0,11.851852,3.0,76.0,2.653846,-0.5,69.0,4.0,0.0,27.0,16.120482,7.0,163.0,4.573171,-3.0,166.0,5.0,0.0,83.0
4,2023,3,8,12,6,20416

In [76]:
flights_historical_performance.shape[0] == flights.shape[0]

True

# Missing Performance Data

In [77]:
route_time_window_counts = flights.groupby(['airline_mkt', 'route_id', 'departure_window']).size().reset_index(name='n_flights')
rare_flights = route_time_window_counts[(route_time_window_counts['n_flights'] <= 5) & (route_time_window_counts['n_flights'] != 0)]
rare_flights

/var/folders/b2/nkpmrw0s42q_zm32rwshljyr0000gn/T/ipykernel_1392/1992945715.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  route_time_window_counts = flights.groupby(['airline_mkt', 'route_id', 'departure_window']).size().reset_index(name='n_flights')


,airline_mkt,route_id,departure_window,n_flights
571,Alaska Airlines Inc.,64,early afternoon,5
573,Alaska Airlines Inc.,64,evening,1
574,Alaska Airlines Inc.,64,night,3
920,Alaska Airlines Inc.,103,morning,1
1002,Alaska Airlines Inc.,112,midday,5
...,...,...,...,...
603365,United Air Lines Inc.,6615,afternoon,4
604012,United Air Lines Inc.,6687,early afternoon,1
604030,United Air Lines Inc.,6689,early afternoon,1
604087,United Air Lines Inc.,6695,night,1


In [78]:
rolling_stats_deduped.isna().sum()

airline_mkt                        0
route_id                           0
departure_window                   0
scheduled_departure_datetime       0
dep_delay_mean_10D              1443
dep_delay_median_10D            1443
dep_delay_max_10D               1443
arr_delay_mean_10D              1500
arr_delay_median_10D            1500
arr_delay_max_10D               1500
cancelled_sum_10D                  0
div_airport_landings_sum_10D       0
n_flights_10D                      0
dep_delay_mean_30D              1222
dep_delay_median_30D            1222
dep_delay_max_30D               1222
arr_delay_mean_30D              1270
arr_delay_median_30D            1270
arr_delay_max_30D               1270
cancelled_sum_30D                  0
div_airport_landings_sum_30D       0
n_flights_30D                      0
dep_delay_mean_90D              1037
dep_delay_median_90D            1037
dep_delay_max_90D               1037
arr_delay_mean_90D              1076
arr_delay_median_90D            1076
a

In [79]:
missing_stats = rolling_stats_deduped[rolling_stats_deduped['arr_delay_mean_10D'].isna()].sort_values('scheduled_departure_datetime')
missing_stats

,airline_mkt,route_id,departure_window,scheduled_departure_datetime,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_mean_30D,arr_delay_median_30D,arr_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,n_flights_30D,dep_delay_mean_90D,dep_delay_median_90D,dep_delay_max_90D,arr_delay_mean_90D,arr_delay_median_90D,arr_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,n_flights_90D
3876874,Frontier Airlines Inc.,5054,overnight,2023-01-01 00:59:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
5732208,Spirit Air Lines,795,overnight,2023-01-01 03:59:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
4346999,Southwest Airlines Co.,372,early morning,2023-01-01 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
4019432,JetBlue Airways,733,early morning,2023-01-01 05:01:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
5918108,Spirit Air Lines,4612,early morning,2023-01-01 05:25:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492432,Allegiant Air,6446,midday,2023-12-27 11:15:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,-10.5,-10.5,-7.0,-34.5,-34.5,-28.0,1.0,0.0,2.0
460001,Allegiant Air,5092,midday,2023-12-27 11:17:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
421097,Allegiant Air,2361,midday,2023-12-27 11:41:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,31.0,31.0,31.0,38.0,38.0,38.0,1.0,0.0,1.0
451765,Allegiant Air,4700,early afternoon,2023-12-27 14:44:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,42.0,42.0,42.0,44.0,44.0,44.0,1.0,0.0,1.0


In [80]:
missing_stats['cancelled_sum_10D'].value_counts()

cancelled_sum_10D
1.0    1338
2.0     132
3.0      24
4.0       3
5.0       3
Name: count, dtype: int64

In [81]:
missing_jan_flights = missing_stats[missing_stats['scheduled_departure_datetime'].dt.month == 1]
missing_jan_flights

,airline_mkt,route_id,departure_window,scheduled_departure_datetime,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_mean_30D,arr_delay_median_30D,arr_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,n_flights_30D,dep_delay_mean_90D,dep_delay_median_90D,dep_delay_max_90D,arr_delay_mean_90D,arr_delay_median_90D,arr_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,n_flights_90D
3876874,Frontier Airlines Inc.,5054,overnight,2023-01-01 00:59:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
5732208,Spirit Air Lines,795,overnight,2023-01-01 03:59:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
4346999,Southwest Airlines Co.,372,early morning,2023-01-01 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
4019432,JetBlue Airways,733,early morning,2023-01-01 05:01:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
5918108,Spirit Air Lines,4612,early morning,2023-01-01 05:25:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114037,American Airlines Inc.,5680,early afternoon,2023-01-10 13:03:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
220407,Alaska Airlines Inc.,5681,afternoon,2023-01-10 15:20:00,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0
2040734,American Airlines Inc.,5291,early morning,2023-01-11 05:10:00,157.0,157.0,157.0,NaN,NaN,NaN,1.0,0.0,1.0,157.0,157.0,157.0,NaN,NaN,NaN,1.0,0.0,1.0,157.0,157.0,157.0,NaN,NaN,NaN,1.0,0.0,1.0
763881,American Airlines Inc.,1157,morning,2023-01-11 07:32:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0


In [82]:
missing_cancelled_flights = missing_stats[missing_stats['cancelled_sum_10D'] != 0]
missing_cancelled_flights.shape

(1500, 31)

In [83]:
# Merge missing cancelled flights with rare flights
rare_missing_cancelled_flights = pd.merge(
    missing_cancelled_flights,
    rare_flights,
    on=['airline_mkt', 'route_id', 'departure_window'],
    how='inner'
)

# Check the result
print(f"Number of missing cancelled flights that are rare: {len(rare_missing_cancelled_flights)}")

Number of missing cancelled flights that are rare: 136


In [84]:
flights[(flights['route_id'] == 3228) & (flights['airline_mkt'] == 'Spirit Air Lines')].sort_values('scheduled_departure_datetime').head(10)

,year,quarter,month,day_of_month,day_of_week,marketing_airline_id,flight_number_marketing_airline,operating_airline_id,tail_number,origin_airport_id,origin_city_market_id,origin,origin_state,dest_airport_id,dest_city_market_id,dest,dest_state,dep_delay,dep_del15,departure_delay_groups,taxi_out,taxi_in,arr_delay,arr_del15,arrival_delay_groups,cancelled,diverted,scheduled_elapsed_time,actual_elapsed_time,air_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,div_airport_landings,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,actual_departure_datetime,actual_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,scheduled_departure_datetime_utc,actual_departure_datetime_utc,scheduled_arrival_datetime_utc,actual_arrival_datetime_utc,is_holiday,dep_delay_clipped,arr_delay_clipped,route_id,hour_of_day,departure_window
5678357,2023,1,1,1,7,20416,2252,20416,N910NK,12889,32211,LAS,NV,11057,31057,CLT,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,246.0,NaN,NaN,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-01-01 05:30:00,2023-01-01 12:36:00,NaN,NaN,1,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-01-01 14:00:00+00:00,NaN,2023-01-01 18:00:00+00:00,NaN,1,NaN,NaN,3228,5,early morning
5678358,2023,1,1,2,1,20416,2252,20416,N925NK,12889,32211,LAS,NV,11057,31057,CLT,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,246.0,NaN,NaN,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-01-02 05:30:00,2023-01-02 12:36:00,NaN,NaN,1,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-01-02 14:00:00+00:00,NaN,2023-01-02 18:00:00+00:00,NaN,1,NaN,NaN,3228,5,early morning
5678359,2023,1,1,5,4,20416,2252,20416,N950NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-3.0,0.0,-1.0,18.0,4.0,-30.0,0.0,-2.0,0,0,247.0,220.0,198.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-01-05 23:29:00,2023-01-06 06:36:00,2023-01-05 23:26:00,2023-01-06 06:06:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-01-06 07:00:00+00:00,2023-01-06 07:26:00+00:00,2023-01-06 12:00:00+00:00,2023-01-06 11:06:00+00:00,0,-3.0,-30.0,3228,23,late night
5678360,2023,1,1,6,5,20416,2252,20416,N947NK,12889,32211,LAS,NV,11057,31057,CLT,NC,5.0,0.0,0.0,17.0,7.0,-20.0,0.0,-2.0,0,0,247.0,222.0,198.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-01-06 23:29:00,2023-01-07 06:36:00,2023-01-06 23:34:00,2023-01-07 06:16:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-01-07 07:00:00+00:00,2023-01-07 07:34:00+00:00,2023-01-07 12:00:00+00:00,2023-01-07 11:16:00+00:00,0,5.0,-20.0,3228,23,late night
5678361,2023,1,1,7,6,20416,2252,20416,N953NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-17.0,0.0,-2.0,14.0,7.0,-29.0,0.0,-2.0,0,0,247.0,235.0,214.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-01-07 23:29:00,2023-01-08 06:36:00,2023-01-07 23:12:00,2023-01-08 06:07:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-01-08 07:00:00+00:00,2023-01-08 07:12:00+00:00,2023-01-08 12:00:00+00:00,2023-01-08 11:07:00+00:00,0,-17.0,-29.0,3228,23,late night
5678362,2023,1,1,8,7,20416,2252,20416,N913NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-6.0,0.0,-1.0,14.0,9.0,-14.0,0.0,-1.0,0,0,247.0,239.0,216.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-01-08 23:29:00,2023-01-09 06:36:00,2023-01-08 23:23:00,2023-01-09 06:22:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-01-09 07:00:00+00:00,2023-01-09 07:23:00+00:00,2023-01-09 12:00:00+00:00,2023-01-09 11:22:00+00:00,0,-6.0,-14.0,3228,23,late night
5678363,2023,1,1,9,1,20416,2252,20416,N934NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-4.0,0.0,-1.0,16.0,4.0,-14.0,0.0,-1.0,0,0,247.0,237.0,217.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-01-09 23:29:00,2023-01-10 06:36:00,2023-01-09 2

In [85]:
rare_missing_cancelled_flights.head()

,airline_mkt,route_id,departure_window,scheduled_departure_datetime,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_mean_30D,arr_delay_median_30D,arr_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,n_flights_30D,dep_delay_mean_90D,dep_delay_median_90D,dep_delay_max_90D,arr_delay_mean_90D,arr_delay_median_90D,arr_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,n_flights_90D,n_flights
0,Spirit Air Lines,4612,early morning,2023-01-01 05:25:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,4
1,Spirit Air Lines,3228,early morning,2023-01-01 05:30:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,2
2,Spirit Air Lines,3228,early morning,2023-01-02 05:30:00,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,2
3,Spirit Air Lines,3964,early morning,2023-01-01 05:45:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,4
4,Spirit Air Lines,1992,morning,2023-01-01 06:15:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,4


In [86]:
missing_stats['scheduled_month'] = missing_stats['scheduled_departure_datetime'].dt.month
missing_stats.groupby('scheduled_month').size()

scheduled_month
1     721
2      26
3      75
4      80
5      36
6     123
7     127
8      55
9     153
10     24
11      5
12     75
dtype: int64

In [87]:
missing_stats.drop(columns='scheduled_month', inplace=True)

In [88]:
# calculate the average aggregated performance by airline_mkt and route_id
route_airline_agg = rolling_stats.groupby(['airline_mkt', 'route_id']).agg({
    'dep_delay_mean_10D': 'mean',
    'dep_delay_median_10D': 'mean',
    'dep_delay_max_10D': 'mean',
    'arr_delay_mean_10D': 'mean',
    'arr_delay_median_10D': 'mean',
    'arr_delay_max_10D': 'mean',
    'cancelled_sum_10D': 'mean',
    'div_airport_landings_sum_10D': 'mean',
    'n_flights_10D': 'mean',
    'dep_delay_mean_30D': 'mean',
    'dep_delay_median_30D': 'mean',
    'dep_delay_max_30D': 'mean',
    'arr_delay_mean_30D': 'mean',
    'arr_delay_median_30D': 'mean',
    'arr_delay_max_30D': 'mean',
    'cancelled_sum_30D': 'mean',
    'div_airport_landings_sum_30D': 'mean',
    'n_flights_30D': 'mean',
    'dep_delay_mean_90D': 'mean',
    'dep_delay_median_90D': 'mean',
    'dep_delay_max_90D': 'mean',
    'arr_delay_mean_90D': 'mean',
    'arr_delay_median_90D': 'mean',
    'arr_delay_max_90D': 'mean',
    'cancelled_sum_90D': 'mean',
    'div_airport_landings_sum_90D': 'mean',
    'n_flights_90D': 'mean'
    
}).reset_index()


In [89]:
agg_cols = route_airline_agg.drop(columns=['airline_mkt', 'route_id']).columns.to_list()
agg_cols

['dep_delay_mean_10D',
 'dep_delay_median_10D',
 'dep_delay_max_10D',
 'arr_delay_mean_10D',
 'arr_delay_median_10D',
 'arr_delay_max_10D',
 'cancelled_sum_10D',
 'div_airport_landings_sum_10D',
 'n_flights_10D',
 'dep_delay_mean_30D',
 'dep_delay_median_30D',
 'dep_delay_max_30D',
 'arr_delay_mean_30D',
 'arr_delay_median_30D',
 'arr_delay_max_30D',
 'cancelled_sum_30D',
 'div_airport_landings_sum_30D',
 'n_flights_30D',
 'dep_delay_mean_90D',
 'dep_delay_median_90D',
 'dep_delay_max_90D',
 'arr_delay_mean_90D',
 'arr_delay_median_90D',
 'arr_delay_max_90D',
 'cancelled_sum_90D',
 'div_airport_landings_sum_90D',
 'n_flights_90D']

In [90]:
# merge the aggregated data back into the missing stats to fill missing values
missing_stats_filled = pd.merge(
    missing_stats,
    route_airline_agg,
    on=['airline_mkt', 'route_id'],
    suffixes=('', '_agg'),
    how='left'
)

# Fill missing values in the original columns with the aggregated values
for col in agg_cols:
    missing_stats_filled[col] = missing_stats_filled[col].fillna(missing_stats_filled[col + '_agg'])

# Drop the extra columns used for merging
missing_stats_filled = missing_stats_filled.drop(columns=[col + '_agg' for col in agg_cols])
missing_stats_filled.head()

,airline_mkt,route_id,departure_window,scheduled_departure_datetime,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_mean_30D,arr_delay_median_30D,arr_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,n_flights_30D,dep_delay_mean_90D,dep_delay_median_90D,dep_delay_max_90D,arr_delay_mean_90D,arr_delay_median_90D,arr_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,n_flights_90D
0,Frontier Airlines Inc.,5054,overnight,2023-01-01 00:59:00,45.978041,42.412037,124.787037,26.233123,20.745370,102.898148,1.0,0.0,0.0,45.000105,39.356481,161.648148,25.119940,17.722222,143.333333,1.0,0.0,0.0,44.229067,37.555556,163.092593,24.994199,15.601852,144.944444,1.0,0.0,0.0
1,Spirit Air Lines,795,overnight,2023-01-01 03:59:00,14.424169,2.728618,78.963816,9.271860,-1.215461,75.250000,1.0,0.0,0.0,13.889934,-0.166118,113.217105,8.680110,-3.427632,112.569079,1.0,0.0,0.0,14.222407,-0.539474,130.098684,9.230187,-3.375000,133.355263,1.0,0.0,0.0
2,Southwest Airlines Co.,372,early morning,2023-01-01 05:00:00,10.196776,4.631175,50.483311,5.273144,-0.052069,47.103471,1.0,0.0,0.0,10.581920,4.002670,78.169559,5.589263,-0.688919,74.131509,1.0,0.0,0.0,11.507480,3.466956,113.375834,6.319452,-1.479306,107.790387,1.0,0.0,0.0
3,JetBlue Airways,733,early morning,2023-01-01 05:01:00,26.741993,12.963499,112.108127,21.657848,9.432163,111.808540,1.0,0.0,0.0,28.040310,11.412534,162.190771,22.906636,8.025138,159.207989,1.0,0.0,0.0,28.753685,11.242080,186.767218,22.891748,7.150826,185.120523,1.0,0.0,0.0
4,Spirit Air Lines,4612,early morning,2023-01-01 05:25:00,17.315171,5.931250,96.131250,8.568672,-1.123438,91.304167,1.0,0.0,0.0,18.165712,4.895313,142.980208,9.692971,-1.181250,137.320833,1.0,0.0,0.0,18.153621,2.626562,170.782292,9.163194,-2.693229,165.992708,1.0,0.0,0.0


In [91]:
missing_stats_filled.isna().sum()

airline_mkt                     0
route_id                        0
departure_window                0
scheduled_departure_datetime    0
dep_delay_mean_10D              8
dep_delay_median_10D            8
dep_delay_max_10D               8
arr_delay_mean_10D              8
arr_delay_median_10D            8
arr_delay_max_10D               8
cancelled_sum_10D               0
div_airport_landings_sum_10D    0
n_flights_10D                   0
dep_delay_mean_30D              8
dep_delay_median_30D            8
dep_delay_max_30D               8
arr_delay_mean_30D              8
arr_delay_median_30D            8
arr_delay_max_30D               8
cancelled_sum_30D               0
div_airport_landings_sum_30D    0
n_flights_30D                   0
dep_delay_mean_90D              8
dep_delay_median_90D            8
dep_delay_max_90D               8
arr_delay_mean_90D              8
arr_delay_median_90D            8
arr_delay_max_90D               8
cancelled_sum_90D               0
div_airport_la

In [92]:
missing_stats_filled[missing_stats_filled['dep_delay_mean_10D'].isna()]

,airline_mkt,route_id,departure_window,scheduled_departure_datetime,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_mean_30D,arr_delay_median_30D,arr_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,n_flights_30D,dep_delay_mean_90D,dep_delay_median_90D,dep_delay_max_90D,arr_delay_mean_90D,arr_delay_median_90D,arr_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,n_flights_90D
542,Allegiant Air,2423,morning,2023-01-03 10:23:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
552,Allegiant Air,6242,early afternoon,2023-01-03 14:44:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
720,United Air Lines Inc.,1284,evening,2023-01-30 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
761,American Airlines Inc.,6590,morning,2023-03-04 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
875,Frontier Airlines Inc.,3986,morning,2023-04-14 08:20:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
1394,American Airlines Inc.,5715,midday,2023-09-24 12:15:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
1437,Frontier Airlines Inc.,4924,evening,2023-12-17 18:24:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
1492,United Air Lines Inc.,2364,afternoon,2023-12-26 15:20:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0


In [93]:
flights[(flights['route_id'] == 1284) & (flights['airline_mkt'] == 'United Air Lines Inc.')].sort_values('scheduled_departure_datetime').head(10)

,year,quarter,month,day_of_month,day_of_week,marketing_airline_id,flight_number_marketing_airline,operating_airline_id,tail_number,origin_airport_id,origin_city_market_id,origin,origin_state,dest_airport_id,dest_city_market_id,dest,dest_state,dep_delay,dep_del15,departure_delay_groups,taxi_out,taxi_in,arr_delay,arr_del15,arrival_delay_groups,cancelled,diverted,scheduled_elapsed_time,actual_elapsed_time,air_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,div_airport_landings,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,actual_departure_datetime,actual_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,scheduled_departure_datetime_utc,actual_departure_datetime_utc,scheduled_arrival_datetime_utc,actual_arrival_datetime_utc,is_holiday,dep_delay_clipped,arr_delay_clipped,route_id,hour_of_day,departure_window
5890501,2023,1,1,30,1,19977,6009,20304,N909SW,11092,31092,CNY,UT,14869,34614,SLC,UT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,52.0,NaN,NaN,183.0,1,NaN,NaN,NaN,NaN,NaN,0.0,1,Moab,Salt Lake City,2023-01-30 17:00:00,2023-01-30 17:52:00,NaN,NaN,0,0,United Air Lines Inc.,SkyWest Airlines Inc.,America/Denver,America/Denver,2023-01-31 00:00:00+00:00,NaN,2023-01-31 01:00:00+00:00,NaN,0,NaN,NaN,1284,17,evening


In [94]:
rolling_stats_not_missing = rolling_stats_deduped.dropna(subset=['arr_delay_mean_10D'])
rolling_stats_deduped_imputed = pd.concat([rolling_stats_not_missing, missing_stats_filled], axis=0)
rolling_stats_deduped_imputed.shape

(7260789, 31)

In [95]:
rolling_stats_deduped.columns.to_list()

['airline_mkt',
 'route_id',
 'departure_window',
 'scheduled_departure_datetime',
 'dep_delay_mean_10D',
 'dep_delay_median_10D',
 'dep_delay_max_10D',
 'arr_delay_mean_10D',
 'arr_delay_median_10D',
 'arr_delay_max_10D',
 'cancelled_sum_10D',
 'div_airport_landings_sum_10D',
 'n_flights_10D',
 'dep_delay_mean_30D',
 'dep_delay_median_30D',
 'dep_delay_max_30D',
 'arr_delay_mean_30D',
 'arr_delay_median_30D',
 'arr_delay_max_30D',
 'cancelled_sum_30D',
 'div_airport_landings_sum_30D',
 'n_flights_30D',
 'dep_delay_mean_90D',
 'dep_delay_median_90D',
 'dep_delay_max_90D',
 'arr_delay_mean_90D',
 'arr_delay_median_90D',
 'arr_delay_max_90D',
 'cancelled_sum_90D',
 'div_airport_landings_sum_90D',
 'n_flights_90D']

In [96]:
rolling_stats[['dep_delay_mean_10D',
 'dep_delay_median_10D',
 'dep_delay_max_10D',
 'arr_delay_mean_10D',
 'arr_delay_median_10D',
 'arr_delay_max_10D',
 'cancelled_sum_10D',
 'div_airport_landings_sum_10D',
 'n_flights_10D']].describe().applymap(lambda x: '{:.2f}'.format(x))

/var/folders/b2/nkpmrw0s42q_zm32rwshljyr0000gn/T/ipykernel_1392/4071758172.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  'n_flights_10D']].describe().applymap(lambda x: '{:.2f}'.format(x))


,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D
count,7259498.00,7259498.00,7259498.00,7259441.00,7259441.00,7259441.00,7260941.00,7260941.00,7260941.00
mean,10.47,1.66,71.59,5.03,-3.01,70.46,0.15,0.01,11.72
std,16.94,13.35,67.04,19.03,16.12,66.79,0.52,0.31,7.53
min,-48.00,-48.00,-48.00,-119.00,-119.00,-119.00,0.00,0.00,0.00
25%,-0.90,-4.50,17.00,-7.22,-12.00,17.00,0.00,0.00,9.00
50%,5.67,-2.00,50.00,1.00,-6.00,49.00,0.00,0.00,10.00
75%,16.82,2.00,111.00,13.00,2.00,110.00,0.00,0.00,11.00
max,209.00,209.00,209.00,207.00,207.00,207.00,15.00,27.00,100.00


In [97]:
rolling_stats_deduped.shape

(7260789, 31)

In [98]:
flights_historical_performance_imputed = pd.merge(flights, rolling_stats_deduped_imputed, 
    on=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime'], 
    how='left')
flights_historical_performance_imputed.shape

(7260941, 88)

In [99]:
flights_historical_performance.shape

(7260941, 88)

# Save to CSV

In [100]:
flights_historical_performance.to_csv(DATA_PATH + '/interim/flights_historical_performance.csv', index=False)
flights_historical_performance_imputed.to_csv(DATA_PATH + '/interim/flights_historical_performance_imputed.csv', index=False)